In [1]:
import nbimporter

import sys
sys.path.append('../')

from module import climate

Importing Jupyter notebook from ..\module\climate.ipynb
Importing Jupyter notebook from ..\module\sun_position.ipynb


## E. 日よけ効果係数

### E.1 日よけ効果係数の算出 (仕様書6.1 式(2)準拠)

- $\gamma_{dsr,j,x+}$：窓等$j$に対して太陽が$x+$側に位置する際の直達日射に対する日よけ効果係数(式$(13)$)
- $\gamma_{dsr,j,x-}$：窓等$j$に対して太陽が$x-$側に位置する際の直達日射に対する日よけ効果係数(式$(17)$)
- $\gamma_{isr,j,y+}$：天空日射に対する日よけ効果係数(式$(21)$) → C.3参照
- $\gamma_{isr,j,y-}$：反射日射に対する日よけ効果係数(式$(24)$) → C.5参照
- $\eta_{j,d,t}$：日付$d$、時刻$t$における窓等$j$の入射角特性 → D.6参照
- $S_{D,d,t}$：日付$d$、時刻$t$における法線面直達日射量$[kcal/m^2]$
- $S_{S,d,t}$：日付$d$、時刻$t$における水平面天空日射量$[kcal/m^2]$
- $\eta_{isr}$：天空・反射日射に対する入射角特性(遮蔽なしの場合)$[-]$
  - 緑本、仕様書では$\;\eta_{isr} = 0.808\;$
  
  
- 式(2)のうち、定数$0.5$は垂直面からみた天空・地表面の形態係数、$0.1$は地表面における日射反射率である。

$$ \begin{align}
\gamma_{wind,j} 
= \big[&\hspace{4pt}\gamma_{dsr,j,x+} \times \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} 
        (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
  + &\hspace{4pt} \gamma_{dsr,j,x-} \times \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} 
        (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
  + &\hspace{4pt}\gamma_{isr,j,y+} \times \sum (\eta_{isr} \times 0.5 \times S_{S,d,t} \\
  + &\hspace{4pt}\gamma_{isr,j,y-} \times \sum \{\eta_{isr} \times 0.1 \times 0.5 
        \times (S_{S,d,t} + S_{D,d,t} \sin h_{s,d,t}) \} \big] \\
\big/ \hspace{4pt} \big[&\sum_{-90<A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
    + &\hspace{4pt}\sum \{\eta_{isr} \times 0.5 \times S_{S,d,t} 
        +  \eta_{isr} \times 0.1 \times 0.5 \times (S_{S,d,t} + S_{D,d,t} \sin h_{s,d,t}) \} \big] \qquad (2)
\\
\end{align} $$

- $A_{wind,j}$：窓等$j$の面積($=x_2y_2$)$[m^2]$
- $A_{wind,j,x+,d,t}$：太陽が$x+$側に位置する日付$d$、時刻$t$において窓等$j$の直達日射が当たる部分の面積$[m^2]$
- $A_{wind,j,x-,d,t}$：太陽が$x-$側に位置する日付$d$、時刻$t$において窓等$j$の直達日射が当たる部分の面積$[m^2]$

$$ \begin{align}
\gamma_{dsr,j,x+} 
= \big\{ & \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} (A_{wind,j,x+,d,t}\hspace{2pt}  
        S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \big\} \\
\big/ \big\{ &\hspace{2pt} A_{wind,j} \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t})  \} \big\} \qquad (13)
\\
\end{align} $$

$$ \begin{align}
\gamma_{dsr,j,x-} 
= \big\{ & \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (A_{wind,j,x-,d,t} \hspace{2pt} 
        S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \big\} \\
\big/ \big\{ &\hspace{2pt} A_{wind,j} \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t})  \} \big\} \qquad (17)
\\
\end{align} $$



### E.2 式(2)分子分母中の時刻$t$における各成分の計算

In [2]:
""" 直達成分の分子分母 """
def calc_dendsr00(Sddhm, etajdt, costheta, Awj): #(2)式分母中直達成分×窓面積
    
    return Awj * Sddhm * etajdt * costheta


def calc_numdsr00(Sddhm, etajdt, costheta, Ax): #(2)式分子中直達成分×窓面積←(13)(17)式

    return Ax * Sddhm * etajdt * costheta


""" 天空成分の分子分母 """
def calc_denisryp00(Ssdhm, etaisr, Awj): #(2)式分母中天空成分×窓面積
    
    return Awj * etaisr * 0.5 * Ssdhm


def calc_numisryp00(Ssdhm, etaisr, Awj, gammayp): #(2)式分子中天空成分×窓面積

    return gammayp * Awj * etaisr * 0.5 * Ssdhm


""" 反射成分の分子分母 """
def calc_denisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj): #(2)式分母中反射成分×窓面積
    
    return Awj * etaisr * 0.1 * 0.5 * (Ssdhm + Sddhm * sinh)


def calc_numisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj, gammaym): #(2)式分子中反射成分×窓面積

    return gammaym * Awj * etaisr * 0.1 * 0.5 * (Ssdhm + Sddhm * sinh)


""" 分子分母それぞれの和 """
def calc_SCF00(Sddhm, Ssdhm, etajdt, etaisr, costheta, sinh, Awj, Ax, gammayp, gammaym):
    
    SCF00 = [0,0]
    
    # SCF00[0]:(2)式の分子への加算分(窓面積をかけた値として)
    SCF00[0] = ( calc_numdsr00(Sddhm, etajdt, costheta, Ax)
               + calc_numisryp00(Ssdhm, etaisr, Awj, gammayp)
               + calc_numisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj, gammaym) )
    # SCF00[1]:(2)式の分母への加算分(窓面積をかけた値として)
    SCF00[1] = ( calc_dendsr00(Sddhm, etajdt, costheta, Awj)             
               + calc_denisryp00(Ssdhm, etaisr, Awj)
               + calc_denisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj) )

    return SCF00


### E.3 式(2)分子分母の期間積算処理

- 式(2)中の時刻$t$における分子および分母を期間積算する。
  - 暖房期、冷房期、非空調期
  - 各月毎
  - 各月、各時刻毎
    - 時刻毎は正時$\pm30$分としたいが、本ツール内でのデータの取り扱いからずれる。前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の間を正時の積算値として計上する。
    

- `SCF[h][i][j]`に格納して積算
  - `h`$=0$：分子, `h`$=1$：分母, `h`$=2$：効果係数
  - `i`$=-2$：冷房期積算, `i`$=-1$：暖房期積算, `i`$=0$：非空調期積算, `i`$=1～12$：各月で積算
  - `j`$=-1$：日積算, `j`$=0～24$：各時刻で積算  

  



In [3]:
""" 期間積算処理 → 日よけ効果係数算出 """

def Output_ShadingCorrectionFactor(c, NDT, SCF01):

    SCF = [[[0 for j in range(-1,25)] for i in range(-2,13)] for h in range(0,3)] 
        # SCF[h][i][j]に格納して積算
        # h = 0：分子, h = 1：分母, h = 2：効果係数
        # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
        #         i = 1～12：各月で積算
        # j = -1：日積算, j = 0～24：各時刻で積算  
        
    """ +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 1時間のループ ++++ """       
    for Hour00 in range(8760):      # ← 12/31 24時は回さないのでHour00=8759がループの最後
        
        """ 日時の計算(A.1) """
        [NDay,NHour] = climate.calc_NDayNHour(Hour00)           
        
        """ 「月」の計算(D.2) """
        Month = climate.calc_Month(c.get_sun_data(Hour00).day_time)        
 
        """ +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ (1/NDT)時間のループ ++++ """ 
            # MM:1時間の内の(1/NDT)間隔の順番, 正時がMM=0, MM=0～NDT-1
        for MM in range(NDT):
            """ 時刻TTの計算(A.1) """
                # TT:時間分割MM毎の時刻[hour]
            TT = climate.calc_TT(NHour, NDT, MM)
            
            """ 時刻Hour01の計算(A.1) """
                # Hour01:前時刻のMM=NDT/2～同時刻のMM=NDT/2-1の時間分割を、その正時に属するものとして
                #        扱うための時刻 → 効果係数算定のための積算用
            Hour01 = int(TT + 0.5)
            
            """ 暖冷房期間のタグ """            
            HCTag = -c.get_sun_data(Hour00).hc_period

            """ 分子分母の期間,月,時間毎の積算(E.3) """
            for h in range(0, 2):
                SCF[h][HCTag][Hour01] += SCF01[h][Hour00][MM]
                SCF[h][HCTag][-1]     += SCF01[h][Hour00][MM]        
                SCF[h][Month][Hour01] += SCF01[h][Hour00][MM]
                SCF[h][Month][-1]     += SCF01[h][Hour00][MM]                      
                    # SCF[h][i][j]に格納して積算
                    # h = 0：分子, h = 1：分母, h = 2：効果係数
                    # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
                    #         i = 1～12：各月で積算
                    # j = -1：日積算, j = 0～24：各時刻で積算             
                    
    """ 期間,月,時間毎の日よけ効果係数算出(E.1) """
    for i in range(-2,13):
        for j in range(-1,25):
            if SCF[1][i][j] != 0:
                SCF[2][i][j] = SCF[0][i][j] / SCF[1][i][j]
            else:
                SCF[2][i][j] = 0

    return SCF

